# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 15
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr15.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr15.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 187 187

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000273749,ENSG00000275835,ENSG00000182636,ENSG00000273173,ENSG00000114062,ENSG00000206190,ENSG00000128731,ENSG00000034053,ENSG00000198690,ENSG00000166912
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000273749,1.00000000,0.03680559,-0.04001166,-0.13993255,0.06297798,-0.03815576,0.00000000,0.000000000,0,0
ENSG00000275835,0.03680559,1.00000000,-0.03581534,0.07465613,-0.08560826,0.09801586,0.00000000,0.000000000,0,0
ENSG00000182636,-0.04001166,-0.03581534,1.00000000,-0.04575868,-0.03163355,0.14242280,-0.22863755,0.000000000,0,0
ENSG00000273173,-0.13993255,0.07465613,-0.04575868,1.00000000,-0.08559488,-0.10763051,-0.22807048,0.005408455,0,0
ENSG00000114062,0.06297798,-0.08560826,-0.03163355,-0.08559488,1.00000000,-0.02026163,0.12739123,0.111127754,0,0
ENSG00000206190,-0.03815576,0.09801586,0.14242280,-0.10763051,-0.02026163,1.00000000,0.02920624,-0.111239262,0,0


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 12

[1] -0.01710902 -0.11491540 -0.13174648 -0.18223295 -0.26035514 -0.28383743
 [7] -0.45943257 -0.48957782 -0.70124661 -0.89806695 -1.66846998 -1.71380993

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 187 187

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000273749,ENSG00000275835,ENSG00000182636,ENSG00000273173,ENSG00000114062,ENSG00000206190,ENSG00000128731,ENSG00000034053,ENSG00000198690,ENSG00000166912
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000273749,1.00000000,0.03680559,-0.04001166,-0.13993255,0.06297798,-0.03815576,0.00000000,0.000000000,0,0
ENSG00000275835,0.03680559,1.00000000,-0.03581534,0.07465613,-0.08560826,0.09801586,0.00000000,0.000000000,0,0
ENSG00000182636,-0.04001166,-0.03581534,1.00000000,-0.04575868,-0.03163355,0.14242280,-0.22863755,0.000000000,0,0
ENSG00000273173,-0.13993255,0.07465613,-0.04575868,1.00000000,-0.08559488,-0.10763051,-0.22807048,0.005408455,0,0
ENSG00000114062,0.06297798,-0.08560826,-0.03163355,-0.08559488,1.00000000,-0.02026163,0.12739123,0.111127754,0,0
ENSG00000206190,-0.03815576,0.09801586,0.14242280,-0.10763051,-0.02026163,1.00000000,0.02920624,-0.111239262,0,0


,ENSG00000273749,ENSG00000275835,ENSG00000182636,ENSG00000273173,ENSG00000114062,ENSG00000206190,ENSG00000128731,ENSG00000034053,ENSG00000198690,ENSG00000166912
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000273749,1.0000000000,-0.010302025,-0.0202118094,-0.0582101647,0.03883479,0.0007286952,0.033355074,0.003050909,-0.0055514452,0.001054966
ENSG00000275835,-0.0103020252,1.000000000,-0.0071489487,0.0226797609,-0.04579499,0.0292642440,-0.007052471,-0.023918898,-0.0009338839,0.003076497
ENSG00000182636,-0.0202118094,-0.007148949,1.0000000000,-0.0009924236,-0.03051131,0.0633706196,-0.088755194,-0.056853178,0.0060018097,-0.031808732
ENSG00000273173,-0.0582101647,0.022679761,-0.0009924236,1.0000000000,-0.06824437,-0.0490050907,-0.096860703,-0.026653514,0.0079429428,-0.030687414
ENSG00000114062,0.0388347883,-0.045794986,-0.0305113114,-0.0682443669,1.00000000,-0.0236006701,0.086614056,0.064466515,-0.0066573088,0.021071765
ENSG00000206190,0.0007286952,0.029264244,0.0633706196,-0.0490050907,-0.02360067,1.0000000000,-0.008560475,-0.047811026,-0.0006977627,0.004138691


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)